In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/KaiserHealthNews.txt'
dest_url = './run_data/kaiserhealth_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# (http:\/\/)(www)?([A-Za-z0-9\.\/]+)(\.[a-zA-Z]+)$ --> Image
# (http(s)?:\/\/(www.)?[A-Za-z0-9\/.?=_-]+(jpg|png)$)
# (http(s)?:\/\/(www.)?([A-Za-z0-9\/.?=_-]+)(jpg|png)$)
# (http(s)?:\/\/)(www.)?(youtu).([A-Za-z0-9\/.?=_-]+) --> Youtube
# (https:\/\/youtu[A-Za-z0-9\/.?=_-]+) -> youtu.be
# (http:\/\/www\.youtube.com[A-Za-z0-9\/.?=_-]+) -> youtube full link

regex_additional_links = r'(http(s)?:\/\/(www.)?[A-Za-z0-9\/.?=_-]+(jpg|png)$)'

In [3]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting..."


In [4]:
# -------------------------------------------------
frame.shape

(3506, 3)

In [5]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'A new sort of extracurricular activity: seeing patients. http://khne.ws/1ceRuaZ '

In [6]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
print("TITLE:\n\n ")
frame['title'] = frame['title'].map(split_txt_form_url)
# --> Get external link and strip it within title
# frame['ext_url'] = frame['title'].map(lambda row: get_url(regex_additional_links[0],row))
# frame['ext_url'] = frame['title'].map(lambda row: get_extend_link(regex_additional_links, row))
# frame['title'] = frame['title'].map(lambda row: row.replace(frame['ext_url'],'') if len(frame['ext_url']) else row)

http://khne.ws/1ceRuaZ
http://khne.ws/1NfQjsp
http://khne.ws/1Do7UrB
http://khne.ws/1y8aJwC
http://khne.ws/1HUROpc
http://khne.ws/1JowAkr
http://khne.ws/1NaiTex
http://khne.ws/1JowAkr
http://cnn.it/1yVmb9H
http://wamu.fm/1NQYhmG
http://kaiserhealthnews.org/news/khn-video-tax-deadline-meets-the-health-law/
http://kaiserhealthnews.org/news/rule-proposed-on-providing-mental-health-parity-in-medicaid-program/
http://kaiserhealthnews.org/news/battle-for-mental-health-parity-produces-mixed-results/
http://kaiserhealthnews.org/news/high-deductible-health-plans-can-ruin-finances/
http://kaiserhealthnews.org/morning-breakout/billionaires-harness-money-technology-in-pursuit-of-fountain-of-youth/
http://kaiserhealthnews.org/news/ritual-not-science-keeps-the-annual-physical-alive/
http://kaiserhealthnews.org/news/medicare-is-stingy-in-first-year-of-doctor-bonuses/
http://kaiserhealthnews.org/news/cancer-survivor-worries-about-supreme-court-ruling-on-obamacare-subsidies/
http://kaiserhealthnews.org

http://khne.ws/1iPxhsH
http://khne.ws/1lzfdTz
http://khne.ws/1lzfdTz
http://khne.ws/1lL8d3h
http://khne.ws/1kmAGRo
http://khne.ws/KtrnyN
http://khne.ws/Kw2EdO
http://khne.ws/1dqmJc1
http://Healthcare.gov.
http://khne.ws/Kw2EdO
http://khne.ws/1i2WE7s
http://khne.ws/1crknfi
http://khne.ws/1i2qid1
http://n.pr/1dgHyqk
http://khne.ws/Knq6tI
http://khne.ws/1dgWQLF
http://khne.ws/1etlr2b
http://khne.ws/19DxdJY
http://khne.ws/1g3TWxd
http://khne.ws/1hYyR8F
http://khne.ws/1ddtUEk
http://khne.ws/KWvkXC
http://khne.ws/1l4axVr
http://khne.ws/HKMauB
http://nbcnews.to/1acYIUT
http://khne.ws/KWvkXC
http://khne.ws/1cuicsr
http://khne.ws/1ade4sD
http://khne.ws/19IYRTD
http://khne.ws/19IYRTD
http://khne.ws/1dyQatO
http://khne.ws/19IYRTD
http://khne.ws/Jld0vs
http://khne.ws/1c1DOv3
http://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/sharedsavingsprogram/Downloads/2014-ACO-Contacts-Directory.pdf
http://khne.ws/1eA7OTf
http://khne.ws/197UjlV
http://khne.ws/JPmnVr
http://khne.ws/1dsrPG5
http://khne

http://khne.ws/YJjWZO
http://twitpic.com/c0viin
http://khne.ws/11JfEhf
http://khne.ws/HKMauB
http://khne.ws/YJjWZO
http://khne.ws/11ttUuy
http://khne.ws/14vmWIM
http://khne.ws/U986qp
http://khne.ws/11ttUuy
http://khne.ws/UJYyAc
http://khne.ws/TlD3pL
http://khne.ws/HKMauB
http://khne.ws/YlTfXj
http://khne.ws/U39URZ
http://khne.ws/14txBUp
http://khne.ws/HKMauB
http://khne.ws/Wg15mB
http://ow.ly/hgWxQ
http://khne.ws/WR6Ch8
http://khne.ws/We5uGD
http://khne.ws/Vo4hsv
http://khne.ws/10IPta2
http://khne.ws/VWyH6X
http://khne.ws/HKMauB
http://khne.ws/We5uGD
http://khne.ws/XIjvIC
http://khne.ws/YCdXWD
http://khne.ws/YC7rPD
http://www.kaiserhealthnews.org/Daily-Report.aspx
http://khne.ws/XI1r1e
http://khne.ws/HKMauB
http://khne.ws/Vj15hP
http://khne.ws/WmeqsS
http://khne.ws/VrVdIr
http://khne.ws/WlxhnD
http://khne.ws/HKMauB
http://khne.ws/WldOnC
http://huff.lv/10JusvY
http://khne.ws/Vyspdw
http://khne.ws/10IPta2
http://khne.ws/SMdLRi
http://khne.ws/HKMauB
http://khne.ws/Y1GRvp
http://khne.ws/Vg

Starting your #Thanksgiving early? LAUGH: http://khne.ws/1vO7Ijo


http://khne.ws/1vO7Ijo
Thinking about dropping employees from your health plan and giving them $ to buy their own? Better read this first: http://khne.ws/1Fp5N3Z


http://khne.ws/1Fp5N3Z
Administration Warns Employers: Dont Dump Sick Workers From Plans: http://khne.ws/1Fp5N3Z


http://khne.ws/1Fp5N3Z
Your #Obamacare plan might auto-renew. Here's why you should take a look at it anyway: http://khne.ws/1yT42Kw


http://khne.ws/1yT42Kw
Obama Officials Seek To Clarify Abortion Coverage Rules: http://khne.ws/1xAu9n2


http://khne.ws/1xAu9n2
HHS Secretary calls for "increased transparency": http://khne.ws/1ySg32A


http://khne.ws/1ySg32A
What's That Meal Going To Mean For Your Waistline? Calorie Counts Coming http://khne.ws/11UznkI


http://khne.ws/11UznkI
Worried about #Calories? Our haiku today has something to say on the matter: http://khne.ws/1uByAMv


http://khne.ws/1uByAMv
Unfriendly skies? http://khne.ws/11QtLIi


http

New Hampshire became the 26th state today to embrace Medicaid expansion, reports @philgalewitz: http://khne.ws/1k9XsXL


http://khne.ws/1k9XsXL
Ive never seen data on how many closely held businesses there are out there. http://khne.ws/1iTO6o1


http://khne.ws/1iTO6o1
ICYMI:  What exactly is a "closely held for-profit corporation"? http://khne.ws/1iTO6o1


http://khne.ws/1iTO6o1
American College of Physicians reverses guidance on pelvic exams in well-woman visits. @JennyAGold has the details. http://khne.ws/1pSZP54


http://khne.ws/1pSZP54
ICYMI: Does the #scotus have final say on contraception coverage for employees? Maybe not, reports @jrovner: http://khne.ws/1x7m634


http://khne.ws/1x7m634
Does the #scotus have final say on contraception coverage for employees? Maybe not, reports @jrovner: http://khne.ws/1x7m634 #hobbylobby


http://khne.ws/1x7m634
Will the #SCOTUS contraceptive decision have a broad or narrow impact? @JayHancock1 examines: http://khne.ws/1iTO6o1


http://khne.ws/1

http://khne.ws/1ctrBgJ
@TEDMED Thanks for sending along!


States Accelerate Moving Nursing Home Residents Into Medicaid Managed Care -- Here's what's happening in Florida: http://khne.ws/1gqgMz0


http://khne.ws/1gqgMz0
.@heidi_demarco chronicles "The Rise Of Pharmacy Clinics" WATCH: http://khne.ws/NyKObu


http://khne.ws/NyKObu
Pharmacists Increasingly Take On Clinical Roles, @annagorman reports: http://khne.ws/1gpYRIK


http://khne.ws/1gpYRIK
Why are some same-sex couples being denied #Obamacare plans? It may be where they live, says @mandrews110: http://khne.ws/1lvbUM4


http://khne.ws/1lvbUM4
Family Planning Clinics In California Squeezed By Health Law: http://khne.ws/1h5dNAE #CA #plannedparenthood


http://khne.ws/1h5dNAE
MT @Julie_appleby: Some getting health coverage subsidies will have to pay up to 9.5% toward premium: http://khne.ws/1bjJgge


http://khne.ws/1bjJgge
Arkansas's Medicaid expansion is on the ropes -- here's what it could mean for #Obamacare: http://khne.ws/1eJNTA

What's hot? Our cartoon: http://khne.ws/19iQDMG


http://khne.ws/19iQDMG
In Kentucky, Confusion And Misinformation Abound On Eve Of Obamacare Rollout http://khne.ws/1eUHulz


http://khne.ws/1eUHulz
Three Critical Measures Of Marketplaces Impact Could Take Several Years To Assess http://khne.ws/15Gh0fe


http://khne.ws/15Gh0fe
RT @jordanrau: How Much Premium Variation Is There Within Your State? See @khnews' Chart of 36 States. http://www.kaiserhealthnews.org/Stories/2013/September/30/premium-variation-intrastate-obamacare-marketplaces-exchanges.aspx


http://www.kaiserhealthnews.org/Stories/2013/September/30/premium-variation-intrastate-obamacare-marketplaces-exchanges.aspx
Poll Finds Most Unaware of Marketplace Opening Date. Hint: It's tomorrow. http://khne.ws/1hfduOd


http://khne.ws/1hfduOd
Latest: Battle Over Health Law Brings Gov't To Brink Of Shutdown http://khne.ws/169B11D


http://khne.ws/169B11D
Where You Live Determines How Much You Pay For Health Insurance, @Julie_Appleby an

High Court's #DOMA Ruling Will Impact Health Benefits For Gay Couples, today's headlines: http://khne.ws/HKMauB


http://khne.ws/HKMauB
RT @scotthensley: I liked @jayhancock1's Kaiser post so much, I ran it on Shots. Test-driving the Obamacare software: http://n.pr/16BB97L


http://n.pr/16BB97L
.@JayHancock1 test drives the Obamacare software: http://khne.ws/10nfi2Z


http://khne.ws/10nfi2Z
Today's cartoon: Can't Chute Straight? http://khne.ws/138MX4g


http://khne.ws/138MX4g
Hospital Officials Complain About Medicare Efforts On Observation Care, @MaryAgnesCarey reports: http://khne.ws/17CMmtm


http://khne.ws/17CMmtm
Grassley: Who Approved These Hospital CEO Bonuses? http://khne.ws/1377jcZ


http://khne.ws/1377jcZ
What's next? @mandrews110 on #DOMA and health coverage for same-sex couples: http://khne.ws/151I64g


http://khne.ws/151I64g
RT @posthealthsci: Scotus strikes down DOMA. Ruling on same-sex marriage might eliminate health care inequities http://wapo.st/1cmlV7P


http://wapo.s

How much will hospitals reduce prices in an effort to win business on insurance exchanges? http://khne.ws/145ltH2


http://khne.ws/145ltH2
The health law doesn't require states to rate insurance plans until 2016. http://khne.ws/XJf5Cy


http://khne.ws/XJf5Cy
Some states will have ratings for the insurance plans in their exchanges by launch on Oct. 1. http://khne.ws/XJf5Cy


http://khne.ws/XJf5Cy
Today's cartoon: Ready for my closeup?  http://khne.ws/YCyrZ3


http://khne.ws/YCyrZ3
Would you text your doctor? @anrao looks at some options: http://khne.ws/XGOiGN


http://khne.ws/XGOiGN
Today's cartoon: Passing on the left? http://khne.ws/Y3jthF


http://khne.ws/Y3jthF
Camp can't say where money to pay for the 'doc fix' would come from, but promises it won't add to deficit: http://khne.ws/13Xk84W


http://khne.ws/13Xk84W
W&amp;M Chairman Dave Camp says fixing how Medicare pays doctors is on his to-do list, @MaryAgnesCarey reports http://khne.ws/13Xk84W


http://khne.ws/13Xk84W
Americans Unc

http://khne.ws/QJplwt
Today's cartoon: Good Heavens! http://khne.ws/T3H1S2


http://khne.ws/T3H1S2
How computer delays may make it difficult for exchanges to open on time, @Julie_Appleby reports:  http://khne.ws/UGtIKj


http://khne.ws/UGtIKj
RT @jrovner: Actually, @dwebbKHN we got Indiana already, so it's 9 states we're waiting on #healthexchange


RT @dwebbkhn: We've got just 10 states still ... in decision-making mode as of last night, @philgalewitz reported #healthexchange


Follow along with @NPRHealth's Twitter chat on health insurance exchanges using #healthexchange w/ @CPRwhitney @afrakt @jrovner


RT @nprhealth: Chat at noon. Deadline looms for states to decide. @philgalewitz and @AlvinHTran report: http://khne.ws/QJ5M7C #healthexchange


http://khne.ws/QJ5M7C
Deadline looming, states count down to health exchange decisions, @PhilGalewitz and @AlvinHTran report: http://khne.ws/QJ5M7C


http://khne.ws/QJ5M7C
Today's cartoon: The Healthy Wait? http://khne.ws/PUxr4e


http://khne

In [7]:
frame.head()

,tweetid,date,title,url
0,586260160462589954,Thu Apr 09 20:11:29 +0000 2015,A new sort of extracurricular activity: seeing...,http://khne.ws/1ceRuaZ
1,586238751334125569,Thu Apr 09 18:46:24 +0000 2015,Houston firefighters have another tool at thei...,http://khne.ws/1NfQjsp
2,586181878635298817,Thu Apr 09 15:00:25 +0000 2015,Will #Montana expand its #Medicaid program? So...,http://khne.ws/1Do7UrB
3,586159308745920512,Thu Apr 09 13:30:44 +0000 2015,Rand Paul's campaign first day: abortion and t...,http://khne.ws/1y8aJwC
4,586159071264432128,Thu Apr 09 13:29:47 +0000 2015,"Enroll America, A Group Dedicated To Promoting...",http://khne.ws/1HUROpc


In [8]:
list_scrapt = ['//div[@class="article-body"]/p/text()',
               '',
               '//div[@class="meta"]/p/span/a/text()'
               ]

In [9]:
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="article-body"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class="meta"]/p/span/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("==========================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

['ERRORS'] 404 , at http://khne.ws/1ceRuaZ
['ERRORS'] 404 , at http://khne.ws/1NfQjsp
['ERRORS'] 404 , at http://khne.ws/1Do7UrB
['ERRORS'] 404 , at http://khne.ws/1y8aJwC
['ERRORS'] 404 , at http://khne.ws/1HUROpc
['ERRORS'] 404 , at http://khne.ws/1JowAkr
['ERRORS'] 404 , at http://khne.ws/1NaiTex
['ERRORS'] 404 , at http://khne.ws/1JowAkr
Header:   200
http://cnn.it/1yVmb9H
0
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  0
tags  :  0
author  :  0



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://wamu.fm/1NQYhmG
4763
0
contents  :  4763
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/khn-video-tax-deadline-meets-the-health-law/
1462
0
contents  :  1462
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/rule-proposed-on-providing-mental-health-parity-in-medicaid-program/
4839
0
contents  :  4839
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/battle-for-mental-health-parity-produces-mixed-results/
3937
0
contents  :  3937
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/high-deductible-health-plans-can-ruin-finances/
9278
0
contents  :  9278
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/morning-breakout/billionaires-harness-money-technology-in-pursuit-of-fountain-of-youth/
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://kaiserhealthnews.org/news/ritual-not-science-keeps-the-annual-physical-alive/
4763
0
contents  :  4763
tags  :  

['ERRORS'] 404 , at http://khne.ws/1A44JPw
['ERRORS'] 404 , at http://khne.ws/1BOewji
['ERRORS'] 404 , at http://khne.ws/1vvrMr4
Header:   200
http://n.pr/1BHQXJ2
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/18hAZcq
['ERRORS'] 404 , at http://khne.ws/1LCdcC9
['ERRORS'] 404 , at http://khne.ws/1wh8qGM
['ERRORS'] 404 , at http://khne.ws/1AD7RWn
['ERRORS'] 404 , at http://khne.ws/1BHezgT
['ERRORS'] 404 , at http://khne.ws/1JDbXF1
Header:   200
http://Healthcare.gov
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1vlMDwX
['ERRORS'] 404 , at http://khne.ws/1zTpmOa
['ERRORS'] 404 , at http://khne.ws/1F3sKdN
['ERRORS'] 404 , at http://khne.ws/1vkGOzC
['ERRORS'] 404 , at http://khne.ws/1A9Y2Qw
['ERRORS'] 404 , at http://khne.ws/19ceN3r
['ERRORS'] 404 , at http://khne.ws/19ceN3r
Header:   200
http://kaiserhealthnews.org/toward-a-better-end-palliative-care-hospice-and-end-of-life/
1666
0
contents  :  1666
tags  :  0
author  :  0

[

['ERRORS'] 404 , at http://khne.ws/1GQmqX6
['ERRORS'] 404 , at http://khne.ws/13v9olN
['ERRORS'] 404 , at http://khne.ws/1BSz9aX
['ERRORS'] 404 , at http://khne.ws/1qX8Pez
['ERRORS'] 404 , at http://khne.ws/1zWYKQn
['ERRORS'] 404 , at http://khne.ws/134CV4N
['ERRORS'] 404 , at http://khne.ws/1wVY5jh
['ERRORS'] 404 , at http://khne.ws/1A4xvDG
['ERRORS'] 404 , at http://khne.ws/1xp9APW
['ERRORS'] 404 , at http://khne.ws/1stYBh7
['ERRORS'] 404 , at http://khne.ws/1zWYKQn
['ERRORS'] 404 , at http://khne.ws/12VewP6
['ERRORS'] 404 , at http://khne.ws/1uwYsta
['ERRORS'] 404 , at http://khne.ws/1zghTM5
['ERRORS'] 404 , at http://khne.ws/1sgE1qd
['ERRORS'] 404 , at http://khne.ws/1BAJMlg
['ERRORS'] 404 , at http://khne.ws/1AkSv7b
['ERRORS'] 404 , at http://khne.ws/1Ghx7Dy
['ERRORS'] 404 , at http://khne.ws/KHNsignup
['ERRORS'] 404 , at http://khne.ws/1x5N4vl
['ERRORS'] 404 , at http://khne.ws/1BbuaBL
['ERRORS'] 404 , at http://khne.ws/1wdWpiU
['ERRORS'] 404 , at http://khne.ws/12ufqlg
['ERRORS'

...

Header:   200
https://medium.com/
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1lXadWx
['ERRORS'] 404 , at http://khne.ws/1ldL7m0
['ERRORS'] 404 , at http://khne.ws/1qJzKtQ
['ERRORS'] 404 , at http://khne.ws/1pCi0tO
['ERRORS'] 404 , at http://khne.ws/UIDAV3
['ERRORS'] 404 , at http://khne.ws/1uDLuzC
['ERRORS'] 404 , at http://khne.ws/1tWoFDc
['ERRORS'] 404 , at http://khne.ws/1zquQCw
['ERRORS'] 404 , at http://khne.ws/UGTI9z
['ERRORS'] 404 , at http://khne.ws/1pxf3e1
['ERRORS'] 404 , at http://khne.ws/UGdWAq
['ERRORS'] 404 , at http://khne.ws/1nzbBDa
['ERRORS'] 404 , at http://khne.ws/UxfU69
['ERRORS'] 404 , at http://khne.ws/1pUcTGR
['ERRORS'] 404 , at http://khne.ws/1rFAfEL
['ERRORS'] 404 , at http://khne.ws/1riz62h
['ERRORS'] 404 , at http://khne.ws/1jZm9vk
['ERRORS'] 404 , at http://khne.ws/1jZkyWo
['ERRORS'] 404 , at http://khne.ws/1A7F6AY
['ERRORS'] 404 , at http://khne.ws/1kcwC7e
['ERRORS'] 404 , at http://khne.ws/1pd1vY1
['ERRORS'] 404 , at htt

['ERRORS'] 404 , at http://khne.ws/1xyJvvH
['ERRORS'] 404 , at http://khne.ws/1j6YV0r
['ERRORS'] 404 , at http://khne.ws/1j6OCJJ
Header:   200
http://Healthcare.gov
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/TmvuS2
['ERRORS'] 404 , at http://khne.ws/1nmB5Cc
['ERRORS'] 404 , at http://khne.ws/Tiln0A
['ERRORS'] 404 , at http://khne.ws/Sc8wMa
['ERRORS'] 404 , at http://khne.ws/1nlHygx
['ERRORS'] 404 , at http://khne.ws/1kEDk3X
['ERRORS'] 404 , at http://khne.ws/1p9YC9R
['ERRORS'] 404 , at http://khne.ws/1oxhXRk
['ERRORS'] 404 , at http://khne.ws/1pRFM7j
['ERRORS'] 404 , at http://khne.ws/1kCSHtH
['ERRORS'] 404 , at http://khne.ws/1l61sMz
['ERRORS'] 404 , at http://khne.ws/1tI79kq
['ERRORS'] 404 , at http://khne.ws/1ovmgMM
['ERRORS'] 404 , at http://khne.ws/1kLu8FQ
['ERRORS'] 404 , at http://khne.ws/1pPwd8T
['ERRORS'] 404 , at http://khne.ws/1wY0veu
['ERRORS'] 404 , at http://khne.ws/1oSpuxz
['ERRORS'] 404 , at http://khne.ws/1n9c5OB
['ERRORS'] 404 , at 

Header:   200
http://healthcare.gov
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1nCQugJ
['ERRORS'] 404 , at http://khne.ws/1r950iu
['ERRORS'] 404 , at http://khne.ws/1hTuNJ7
['ERRORS'] 404 , at http://khne.ws/1eQzmEE
['ERRORS'] 404 , at http://khne.ws/1hTydfR
['ERRORS'] 404 , at http://khne.ws/1neoMJP
['ERRORS'] 404 , at http://khne.ws/1h7AwWr
['ERRORS'] 404 , at http://khne.ws/1r2TBPi
['ERRORS'] 404 , at http://khne.ws/1j86cfH
['ERRORS'] 404 , at http://khne.ws/1gE3xK7
['ERRORS'] 404 , at http://khne.ws/1nbTuDt
Header:   200
http://bit.ly/1gGoBmN
8855
0
contents  :  8855
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1eQpxkg
['ERRORS'] 404 , at http://khne.ws/1eIWDs0
['ERRORS'] 404 , at http://khne.ws/1l1KroL
['ERRORS'] 404 , at http://khne.ws/1hSEVwL
['ERRORS'] 404 , at http://khne.ws/1t61eIq
['ERRORS'] 404 , at http://khne.ws/1imxNgC
['ERRORS'] 404 , at http://khne.ws/1eLgMbp
['ERRORS'] 404 , at http://khne.ws/QnuiMh
['ERRORS'] 404 , a

['ERRORS'] 404 , at http://khne.ws/1d6khYI
['ERRORS'] 404 , at http://khne.ws/1mw1AHD
['ERRORS'] 404 , at http://khne.ws/1blqDIA
['ERRORS'] 404 , at http://khne.ws/1dNcqDD
['ERRORS'] 404 , at http://khne.ws/1blqDIA
['ERRORS'] 404 , at http://khne.ws/N7hdFk
['ERRORS'] 404 , at http://khne.ws/1mrpJ24
['ERRORS'] 404 , at http://khne.ws/1h1AFg4
['ERRORS'] 404 , at http://khne.ws/1m5AgfM
['ERRORS'] 404 , at http://khne.ws/1m5AgfM
['ERRORS'] 404 , at http://khne.ws/1h12rcD
['ERRORS'] 404 , at http://khne.ws/1dQtbZG
['ERRORS'] 404 , at http://khne.ws/1ezjFug
['ERRORS'] 404 , at http://khne.ws/1ggIPTn
['ERRORS'] 404 , at http://khne.ws/1gv0FBi
['ERRORS'] 404 , at http://khne.ws/1ewAFkR
['ERRORS'] 404 , at http://khne.ws/1gv0FBi
['ERRORS'] 404 , at http://khne.ws/1gv0FBi
['ERRORS'] 404 , at http://khne.ws/1gbk22P
['ERRORS'] 404 , at http://khne.ws/1eX31Jn
['ERRORS'] 404 , at http://khne.ws/1opV0Rz
['ERRORS'] 404 , at http://khne.ws/LX4pjQ
['ERRORS'] 404 , at http://khne.ws/1j5pEf2
['ERRORS'] 40

['ERRORS'] 404 , at http://khne.ws/1cuicsr
['ERRORS'] 404 , at http://khne.ws/1ade4sD
['ERRORS'] 404 , at http://khne.ws/19IYRTD
['ERRORS'] 404 , at http://khne.ws/19IYRTD
['ERRORS'] 404 , at http://khne.ws/1dyQatO
['ERRORS'] 404 , at http://khne.ws/19IYRTD
['ERRORS'] 404 , at http://khne.ws/Jld0vs
['ERRORS'] 404 , at http://khne.ws/1c1DOv3
Header:   200
http://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/sharedsavingsprogram/Downloads/2014-ACO-Contacts-Directory.pdf
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://khne.ws/1eA7OTf
['ERRORS'] 404 , at http://khne.ws/197UjlV
['ERRORS'] 404 , at http://khne.ws/JPmnVr
['ERRORS'] 404 , at http://khne.ws/1dsrPG5
['ERRORS'] 404 , at http://khne.ws/19hABdb
['ERRORS'] 404 , at http://khne.ws/1jpKJoj
['ERRORS'] 404 , at http://khne.ws/1i7y4Vv
['ERRORS'] 404 , at http://khne.ws/JPmnVr
['ERRORS'] 404 , at http://khne.ws/JPmnVr
Header:   200
http://ow.ly/rW4hf
3122
0
contents  :  3122
tags  :  0
author  :  0

['ERRORS

In [ ]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape

In [ ]:
frame.info()

In [ ]:
# test_url= 'http://ow.ly/1Piv0Y'

# res = requests.get(test_url)
# if res.status_code  == 200:
# #     soup = BeautifulSoup(res.content, 'lxml')
# #     author = soup.find("span",{"class":"byline"})
# #     print(author)
# #     content = soup.find("div", {"class": "article-body"})
# #     print(content)

#     # Initialize XPath
#     xpath_selector = Selector(text=res.text)
    
#     author = xpath_selector.xpath('//*[@id="post-22962"]/header/div[2]/p/span/a/strong/text()').extract()[0]
#     print("Author: ", author)
#     print("\n")
#     content = xpath_selector.xpath('//div[@class="article-body"]/p/text()').extract()
#     content = join_txt(content)
#     print(content)
# else:
#     print("[ERROR]: ", res.status_code)

In [ ]:
frame.to_csv(dest_url, sep='|', encoding='utf-8', index=False)

In [ ]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
    success_date = './date/success_date.txt'
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
            f1 = open(success_date, "a")
            f1.write(frame['date'].iloc[key])
            f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

In [ ]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================